#import

In [215]:

import numpy as np
import pandas as pd
import librosa

from sklearn.model_selection import train_test_split
from scipy.stats import skew

from sklearn.multiclass import  OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsOneClassifier


import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LayerNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score

資料前處理

In [ ]:
# 資料判斷
df_csv = pd.read_csv("Training Dataset/training datalist.csv")
print("資料資訊")
df_csv.info()

In [ ]:
# 挑選出要訓練的Disease category 1、2、3
df_csv = df_csv.loc[df_csv['Disease category'].isin([1, 2, 3, 4, 5]), ['ID', 'Disease category']]

# 在dataframe中加入要訓練的音檔路徑
df_csv['wav_path'] = df_csv['ID'].map("./Training Dataset/training_voice_data/{}.wav".format)

print("Disease category in source_df :",df_csv['Disease category'].unique())
print("source_df :\n", df_csv["wav_path"])

In [219]:
# 定義函數
def audio_to_mfccs(filename, sample_rate=44100, offset=0, duration=None):
    # 讀取音訊檔案，並設定取樣率、起始時間、及持續時間
    voice, sample_rate = librosa.load(
        filename, sr=sample_rate, offset=offset, duration=duration
    )

    # 將時間值轉換為 FFT 與 hop length 所需的框架數 (以取樣點表示)
    n_fft = int(16/1000 * sample_rate)  # 將 16 毫秒轉換為取樣點
    hop_length = int(8/1000 * sample_rate)  # 將 8 毫秒轉換為取樣點

    # 計算音訊數據的 MFCC 特徵
    mfcc_feature = librosa.feature.mfcc(
        y=voice, sr=sample_rate, n_mfcc=13, n_fft=n_fft, hop_length=hop_length)

    # 計算 MFCC 的一階和二階差分特徵
    delta_mfcc_feature = librosa.feature.delta(mfcc_feature)

    # 將原始 MFCC 特徵和差分特徵串聯起來，得到所有幀的特徵向量
    mfccs = np.concatenate((mfcc_feature, delta_mfcc_feature))
    mfccs_features = np.transpose(mfccs)  # 將矩陣轉置，使每行代表一個幀

    # 返回特徵向量
    return mfccs_features

將資料切分為1000,6的矩陣


In [ ]:

training_id = df_csv['ID'].tolist()
training_data = pd.DataFrame()
for id in training_id:
    
    mfccs_feature = audio_to_mfccs(df_csv[df_csv['ID']==id]['wav_path'].values[0])
    df = pd.DataFrame()
    for i in range(26):
        df_i = pd.DataFrame(np.array(mfccs_feature[0][i]).reshape(1,-1))
        df = pd.concat([df, df_i], axis=1)
        
    # print("id :",id, ", number of frames :", df.shape[0])
    # 訓練資料標記
    label = df_csv[df_csv['ID']==id]['Disease category'].values[0]
    if label==1:
        df['c1'] = 1; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 0
    elif label==2:
        df['c1'] = 0; df['c2'] = 1; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 0
    elif label==3:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 1 ; df['c4'] = 0; df['c5'] = 0
    elif label==4:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 1; df['c5'] = 0
    elif label==5:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 1
    else:
        df['c1'] = np.nan; df['c2'] = np.nan; df['c3'] = np.nan; df['c4'] = np.nan; df['c5'] = np.nan

    training_data = pd.concat([training_data, df])
    
print("training_data.shape :", training_data.shape)

x_train = training_data.iloc[:, :-5]
y_train = training_data.iloc[:, -5:]
print("x_train.shape, y_train.shape :", x_train.shape, y_train.shape)
print("y_train.columns :", y_train.columns.tolist())


轉成numpy矩陣

In [ ]:
x = x_train.to_numpy()
y = y_train.to_numpy()


y = np.argmax(y, axis=1)

print(y)

將y的0~4類轉成1~5類

In [222]:
for i in range(len(y)):
    y[i] += 1

new_x為記錄x最大最小值等的矩陣


In [223]:
new_x = np.zeros((1000, 6))
for i in range(1000):
    # 計算最大值
    max_a = np.max(x[i])

    # 計算最小值
    min_a = np.min(x[i])

    # 計算平均數
    mean_a = np.mean(x[i])

    # 計算中位數
    median_a = np.median(x[i])

    # 計算標準差
    std_a = np.std(x[i])

    # 計算偏度
    skew_a = skew(x[i])

    # 轉換矩陣為一個包含六個元素的陣列
    new_x[i] = np.array([max_a, min_a, mean_a, median_a, std_a, skew_a])
print(new_x)

[[ 1.48494080e+02 -5.53074524e+02 -1.63179493e+01 -5.11780977e-01
   1.11396294e+02 -4.17364942e+00]
 [ 1.32414246e+02 -2.22018585e+02 -6.04307556e+00 -9.04636979e-01
   5.09200935e+01 -2.15934679e+00]
 [ 1.86702805e+02 -3.39410553e+02 -1.00040083e+01 -6.12908542e-01
   7.62460938e+01 -2.44126632e+00]
 ...
 [ 1.13782791e+02 -2.79279510e+02 -9.72987843e+00 -1.34194613e+00
   5.88886871e+01 -3.32960002e+00]
 [ 1.23644379e+02 -4.17048859e+02 -1.30127821e+01 -2.56308377e-01
   8.47630157e+01 -3.99959885e+00]
 [ 1.93103134e+02 -3.73614166e+02 -9.09355068e+00 -3.75599675e-02
   8.26008835e+01 -2.73785199e+00]]


In [224]:
x_train2, x_test ,y_train2, y_test = train_test_split(new_x, y, test_size=0.8, random_state=42)

clf = OneVsOneClassifier(SVC(kernel="linear"))

clf.fit(x_train2,y_train2)

y_pred = clf.predict(x_test)


In [ ]:
print(y_pred)